In [24]:
import re
import openpyxl
import pandas as pd
from datetime import datetime
from google.oauth2 import service_account
from googleapiclient.discovery import build
from google.oauth2.credentials import Credentials

# Función para extraer todos los números de una cadena
def extract_numbers_from_string(text):
    return [match for match in re.findall(r'\d+', text)]

# API sheets
SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
KEY = 'key.json'
SPREADSHEET_ID = '1k9sNdQcOEsk1VW_k6uiAbJc9XEmYpZhPvl2T4n3b_AQ'
creds = service_account.Credentials.from_service_account_file(KEY, scopes=SCOPES)
service = build('sheets', 'v4', credentials=creds)
sheet = service.spreadsheets()

RANGE_NAME = 'receiptf'
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID, range=RANGE_NAME).execute()
values = result.get('values', [])
df = pd.DataFrame(values[1:], columns=values[0])

cols_to_consider = [col for col in df.columns if col not in ['receipt_date', 'receipt_number']]
df = df.drop_duplicates(subset=cols_to_consider)

df['%_released'] = pd.to_numeric(df['%_released'])

df_grouped = df.groupby(['SPO Number', 'Main SMP'])['%_released'].sum().reset_index()

list_below_100 = df_grouped[df_grouped['%_released'] < 100]
list_above_100 = df_grouped[df_grouped['%_released'] > 100]

print("Archivos históricos cargados\n")

# Archivo Excel local
start = datetime.now()
format_date_time = "%d%m%Y"
excel_file_path = f'./{start.strftime(format_date_time).upper()}/import_ppa_powerbi.xlsx'
spo_column_index = 1  # Columna con los SPO
smp_column_index = 3  # Columna con los SMP
pending_invoiced_column_index = 6  # Columna %_pending_invoiced
hoja_nombre = "query_invoice"  # Nombre de la hoja
wb = openpyxl.load_workbook(excel_file_path)
ws = wb[hoja_nombre]

spo_list = []
smp_list = []

# Recopilar listas de SPO y SMP del archivo Excel
for row_num in range(2, ws.max_row + 1):
    spo_value = extract_numbers_from_string(str(ws.cell(row=row_num, column=spo_column_index).value))
    smp_value = extract_numbers_from_string(str(ws.cell(row=row_num, column=smp_column_index).value))

    if spo_value:
        spo_list.extend(spo_value)
    else:
        break  

    if smp_value:
        smp_list.extend(smp_value)
    else:
        break 

spo_list = [str(i) for i in spo_list]
smp_list = ["SMP-WO-" + i for i in smp_list]

print("SPO y SMP cargados\n")

# Buscar coincidencias y actualizar el porcentaje en el archivo Excel
for i, row in list_below_100.iterrows():
    spo = row['SPO Number']
    smp = row['Main SMP']

    # Buscar la fila correspondiente en el archivo Excel
    for row_num in range(2, ws.max_row + 1):
        excel_spo = extract_numbers_from_string(str(ws.cell(row=row_num, column=spo_column_index).value))
        excel_smp = extract_numbers_from_string(str(ws.cell(row=row_num, column=smp_column_index).value))

        if excel_spo and excel_smp and spo in excel_spo and smp in ["SMP-WO-" + i for i in excel_smp]:
            historical_percentage = row['%_released']
            print(f"Datos encontrados: ({spo}, {smp}) - Porcentaje histórico: {historical_percentage}%")

            # Calcular el porcentaje faltante
            target_percentage = 100
            missing_percentage = target_percentage - historical_percentage
            print(f"Porcentaje faltante: {missing_percentage}%")

            # Actualizar el valor en la columna %_pending_invoiced
            ws.cell(row=row_num, column=pending_invoiced_column_index, value=missing_percentage)

            # Salir del bucle interno una vez que se ha actualizado
            break

print("\nProceso finalizado")

# Guardar los cambios en el libro de Excel
wb.save(excel_file_path)


Archivos históricos cargados

SPO y SMP cargados

Datos encontrados: (47693051, SMP-WO-0199736) - Porcentaje histórico: 90%
Porcentaje faltante: 10%
Datos encontrados: (47761716, SMP-WO-0199758) - Porcentaje histórico: 70%
Porcentaje faltante: 30%
Datos encontrados: (47853531, SMP-WO-0202763) - Porcentaje histórico: 90%
Porcentaje faltante: 10%
Datos encontrados: (47897099, SMP-WO-0202355) - Porcentaje histórico: 30%
Porcentaje faltante: 70%
Datos encontrados: (48357947, SMP-WO-0216519) - Porcentaje histórico: 70%
Porcentaje faltante: 30%
Datos encontrados: (48507175, SMP-WO-0219429) - Porcentaje histórico: 60%
Porcentaje faltante: 40%
Datos encontrados: (48507232, SMP-WO-0217814) - Porcentaje histórico: 70%
Porcentaje faltante: 30%
Datos encontrados: (48507244, SMP-WO-0217825) - Porcentaje histórico: 70%
Porcentaje faltante: 30%

Proceso finalizado
